Hosein Seifi<br>
810100386

#Pre-Processing

##Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/ML_HW1_Data/penguins.csv')

Mounted at /content/drive


##Drop invalid values

In [ ]:
data = data.drop(data.index[data['culmen_length_mm']=='x'].tolist())

##Find Labels and set features names to a list

In [ ]:
labels = list(set(data['species']))
features = ['culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g']

##Create Numeric Labels

In [ ]:
numerical_label = list()
for i in range(data.shape[0]):
    numerical_label.append(labels.index(data.iloc[i]['species']))
data['label'] = numerical_label

##Cast features to float data type

In [ ]:
for col in features:
    data[col] = [float(i) for i in list(data[col])]

#Naive Bayes from scratch

##Split data into train and test sets

In [ ]:
from random import randint
def train_test_split(data, train_size):
    data = data.values.tolist()
    train = list()
    test = list()
    for i in range(len(data)):
        if randint(0,10)/10 > train_size:
            test.append(data[i])
        else:
            train.append(data[i])
    return train, test

In [ ]:
train, test = train_test_split(data, 0.8)
train = pd.DataFrame(train, columns=data.columns)
test = pd.DataFrame(test, columns=data.columns)

##Group train data by their labels

In [ ]:
def group_by_label(data):
    pdata = dict()
    for l in range(len(labels)):
        pdata[l] = list()
    for i in range(data.shape[0]):
        pdata[data.iloc[i]['label']].append(data.iloc[i][features].tolist())
    return pdata

In [ ]:
gtrain = group_by_label(train)

##Calculate priors

In [ ]:
def prior(data, labels, n):
    pci = dict()
    for i in range(len(labels)):
        pci[i] = len(data[i])/n
    return pci

In [ ]:
pci = prior(gtrain, labels, sum([len(i) for i in gtrain.values()]))

##Mean and Standard deviation functions

In [ ]:
def mean(x):
    return (1/len(x))*sum(x)

In [ ]:
def stddev(x):
    m = mean(x)
    x2 = [(i-m)**2 for i in x]
    return (sum(x2)/(len(x)-1))**0.5

##Calculate Mean and Standard deviation for the whole train data

In [ ]:
meanstd = dict()
for col in features:
    a = mean(list(train[col]))
    b = stddev(list(train[col]))
    meanstd[col] = [a,b]

##Calculate Mean and Standard deviation for each class

In [ ]:
classes_meanstd = dict()
for k in gtrain.keys():
    temp = list()
    for i in range(4):
        a = mean([item[i] for item in gtrain[k]])
        b = stddev([item[i] for item in gtrain[k]])
        temp.append([a,b])
    classes_meanstd[k] = temp

In [ ]:
for c, e in classes_meanstd.items():
  print(c)
  for i in e:
    print(i)

0
[38.86434108527132, 2.704995334146056]
[18.30077519379844, 1.2155564649080872]
[189.5658914728682, 6.661274466600205]
[3692.4418604651164, 456.1494393798062]
1
[47.41938775510203, 3.332491039524084]
[14.98979591836734, 1.0345066851560956]
[217.28571428571428, 6.57549339609294]
[5050.2551020408155, 524.9017445851615]
2
[48.74905660377358, 3.1139649868041364]
[18.409433962264153, 1.1755954548397025]
[195.62264150943395, 6.549299478377409]
[3707.5471698113206, 409.49032890043924]


##Gaussian PDF

In [ ]:
pi = 3.14159265359
e = 2.71828
def probability(x, mean, stddev):
    exp = e**( (-0.5) * ( ( (x-mean) /stddev) ** 2) )
    return ( 1 / (stddev * ( 2 * pi ) ** 0.5 ) ) * exp

##Argmax function

In [ ]:
def argmax(a):
    maxx = 0
    argmaxx = 0
    for arg,x in a.items():
        if x > maxx:
            maxx = x
            argmaxx = arg
    return argmaxx

##Calculate probability of each sample belonging to one of three classes

In [ ]:
def calc_prob(classes_meanstd, x):
    prob = dict()
    for k in classes_meanstd.keys():
        temp = 1
        i = 0
        for attr in classes_meanstd[k]:
            temp *= probability(x[i], attr[0], attr[1])
            i += 1
        prob[k] = temp
    return prob

##Seperate label and features of test set

In [ ]:
test = test.values.tolist()
x_test = [ item[1:5] for item in test ]
y_test = [ item[5] for item in test ]

##Predict function

In [ ]:
def predict(classes_meanstd, test):
    predicted = list()
    for x in test:
        prob = calc_prob(classes_meanstd, x)
        print(prob)
        predicted.append(argmax(prob))
    return predicted

##predict label of each test sample

In [ ]:
pred = predict(classes_meanstd, x_test)

{0: 1.5794245836203354e-06, 1: 5.96271541025297e-15, 2: 7.17834423592286e-09}
{0: 1.6396881274309843e-08, 1: 8.583791331948866e-16, 2: 7.761241353970638e-09}
{0: 1.1730750663661745e-06, 1: 1.2493921612142156e-17, 2: 1.202444773312526e-08}
{0: 4.894781974679064e-07, 1: 1.966799587034889e-14, 2: 3.1117462107467204e-10}
{0: 6.730855655512462e-08, 1: 3.293374801197543e-18, 2: 9.506100993065504e-10}
{0: 1.1876302258726626e-06, 1: 2.007027162797925e-16, 2: 3.055389104959004e-10}
{0: 8.104509538736119e-07, 1: 2.6749952985504396e-15, 2: 8.911098758682732e-09}
{0: 2.9400184301256846e-07, 1: 7.585215284150891e-19, 2: 9.429902625025325e-12}
{0: 2.7017913521448186e-07, 1: 2.5546821204364208e-17, 2: 1.3900724385305739e-11}
{0: 9.650181540013596e-07, 1: 6.48744661615235e-18, 2: 3.828994646765941e-10}
{0: 4.998170332740984e-08, 1: 2.414671516869072e-10, 2: 1.536950096684312e-08}
{0: 7.132286518875441e-07, 1: 5.795851421693321e-17, 2: 1.2595358754086641e-10}
{0: 4.821738514469739e-07, 1: 1.49640505358

##Confusion Matrix

In [ ]:
import numpy as np
def confusion_matrix(y, pred):
    cm = np.zeros([3,3])
    for i in range(len(y)):
        cm[y[i],pred[i]] += 1
    return cm

In [ ]:
cm = confusion_matrix(y_test,pred)
cm

array([[21.,  0.,  0.],
       [ 0., 22.,  0.],
       [ 1.,  0., 13.]])

##One vs All confusion Matrix 

In [ ]:
def onevsallcm(cm, c):
    result = np.zeros([2,2])
    result[0,0] = cm[c,c]
    for i in range(3):
        if not i == c:
            result[0,1] += cm[c,i] 
            result[1,0] += cm[i,c]
    result[1,1] = np.sum(cm) - result[0,0] - result[0,1] - result[1,0]
    return result

##Evaluate model

In [ ]:
def evaluate(cm):
    metrics = dict()
    for i in range(cm.shape[0]):
        metrics[i] = dict()
        cmi = onevsallcm(cm,i)
        p = cmi[0,0] / (cmi[0,0] + cmi[1,0])
        r = cmi[0,0] / (cmi[0,0] + cmi[0,1])
        acc = (cmi[0,0]+cmi[1,1]) / np.sum(cmi)
        metrics[i]['Precision'] = p
        metrics[i]['Recall'] = r
        metrics[i]['Accuracy'] = acc
        metrics[i]['Confusion Matrix'] = cmi
    return metrics

In [ ]:
evaluation = evaluate(cm)
for c, e in evaluation.items():
  print("class: ", c)
  for n, v in e.items():
    print(" ", n, ":\n    ", v)
  print("---------------------")

class:  0
  Precision :
     0.9545454545454546
  Recall :
     1.0
  Accuracy :
     0.9824561403508771
  Confusion Matrix :
     [[21.  0.]
 [ 1. 35.]]
---------------------
class:  1
  Precision :
     1.0
  Recall :
     1.0
  Accuracy :
     1.0
  Confusion Matrix :
     [[22.  0.]
 [ 0. 35.]]
---------------------
class:  2
  Precision :
     1.0
  Recall :
     0.9285714285714286
  Accuracy :
     0.9824561403508771
  Confusion Matrix :
     [[13.  1.]
 [ 0. 43.]]
---------------------


#Naive Bayes using Sklearn

##Train model

##Split Data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data[features], data['label'])

In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model = model.fit(X_train, y_train)

##Predict labels for test set

In [ ]:
sklearn_pred = model.predict(X_test)

##Evaluation

In [ ]:
from sklearn.metrics import classification_report
cr = classification_report(y_test, sklearn_pred)

In [ ]:
for line in cr.split('\n'):
  print(line)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        29
           1       1.00      0.94      0.97        17
           2       0.97      1.00      0.99        39

    accuracy                           0.99        85
   macro avg       0.99      0.98      0.99        85
weighted avg       0.99      0.99      0.99        85



##evaluate using my own evaluator

In [ ]:
evaluation = evaluate(confusion_matrix(y_test.tolist(),sklearn_pred.tolist()))
for c, e in evaluation.items():
  print("class: ", c)
  for n, v in e.items():
    print(" ", n, ":\n    ", v)
  print("---------------------")

class:  0
  Precision :
     1.0
  Recall :
     1.0
  Accuracy :
     1.0
  Confusion Matrix :
     [[29.  0.]
 [ 0. 56.]]
---------------------
class:  1
  Precision :
     1.0
  Recall :
     0.9411764705882353
  Accuracy :
     0.9882352941176471
  Confusion Matrix :
     [[16.  1.]
 [ 0. 68.]]
---------------------
class:  2
  Precision :
     0.975
  Recall :
     1.0
  Accuracy :
     0.9882352941176471
  Confusion Matrix :
     [[39.  0.]
 [ 1. 45.]]
---------------------
